In [23]:
import json

f = open('raw_data/rikishi.json')

# returns JSON object as a dictionary
data = json.load(f)
data = data["records"]

In [24]:
import pandas as pd

df = pd.DataFrame(data)

In [25]:
# GET /api/basho/:bashoId/torikumi/:division/:day
base_url = "https://sumo-api.com/api"

first_year = 1958
last_year = 2024
months = ["01", "03", "05", "07", "09", "11"]
first_day = 1
last_day = 15
divisions = ["makuuchi", "juryo", "makushita", "sandanme", "jonidan", "jonokuchi"]

excluded_basho_ids = {"195801", "202407", "202409", "202411"} # tournaments in range that don't have data

years = list(range(first_year, last_year + 1))
days = list(range(first_day, last_day + 1))
days = ['{:02d}'.format(day) for day in days]

params = [(str(year) + month, division, day) for year in years for month in months for day in days for division in divisions]
params = [param for param in params if param[0] not in excluded_basho_ids]

In [32]:
import json
import requests
import time

failed_urls = []
failed_urls_path = "failed_urls.json"
for param in params:
    basho_id, division, day = param
    url = f"{base_url}/basho/{basho_id}/torikumi/{division}/{day}"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            filename = f"raw_data/basho/{basho_id}_{division}_{day}.json"
            with open(filename, 'w') as file:
                json.dump(response.json(), file, indent=2)
        else:
            print(f"Failed to fetch {url}, status code: {response.status_code}")
            failedUrls += [url]

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}, error: {e}")
        failedUrls += [url]

    time.sleep(2)

with open(failed_urls_path, 'w') as file:
    json.dump(failed_urls, file)

print(failed_urls)
print("DONE!")

[]
DONE!
